In [3]:
# import imports
# import importlib
# importlib.reload(imports)
# from imports import *
import torch.nn as nn
import dill
import torch
from storage import Storage

class LSTM(nn.Module):
    def __init__(self, **kwargs):
        super().__init__()

        self.lstm = nn.LSTM(INTERNAL_EMBEDDING_SIZE2, INTERNAL_EMBEDDING_SIZE2 // 2, 
                            num_layers=1, batch_first=True, bidirectional=True)
    def forward(self, x):
        return self.lstm(x)[0]


prefix = "results writers big"
with open(f"{prefix}/some_model_CLASS.dill", "rb") as file:
    Model = dill.load(file)
model = torch.load(f"{prefix}/some_model.pt", map_location=torch.device('cpu')) #).v1", map_location=torch.device('cuda:0'))

with open(f"{prefix}/storage_path.dill", "rb") as file:
    storage_path = dill.load(file)
with open(f"{prefix}/is_infected_test.dill", "rb") as file:
    is_infected = dill.load(file)

storage = Storage(str(storage_path))
params = storage.get_meta("params")

x_test = storage.get_meta("x_test").float()
y_test = storage.get_meta("y_test").float()
text_res = storage.get_meta("text_res_test")
is_infected = storage.get_meta("is_infected_test")

ModuleNotFoundError: No module named 'xformers'

In [4]:
dummy_input = x_test[0:1]
with torch.no_grad():
        print(model(dummy_input))
        # torch.onnx.export(model, dummy_input, "model.v1.onnx", verbose=True)

tensor([[ 7.9433, -7.6244, -3.0897]])


In [7]:
torch_script_graph, unconvertible_ops = torch.onnx.utils.unconvertible_ops(
    model, dummy_input #, opset_version=opset_version
)
unconvertible_ops

================ Diagnostic Run torch.onnx.export version 2.0.0 ================
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



[]

[]

In [9]:
torch.onnx.export(model, (dummy_input, ), f"{prefix}/model.onnx", 
                input_names = ['input'],
                output_names = ['output'])#, verbose=True)
        # print(torch.onnx.export_to_pretty_string(model, (dummy_input, )))#, verbose=True))


================ Diagnostic Run torch.onnx.export version 2.0.0 ================
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [17]:
from inference import torch_model_runner, onnx_model_runner
for a in [dummy_input, np.zeros_like(dummy_input)]:
    print('torch:', torch_model_runner(model)(torch.Tensor(a)))
    print('onnx:', onnx_model_runner(f'{prefix}/model.onnx')(a))

torch: tensor([[ 7.9433, -7.6244, -3.0897]])
onnx: [[ 7.943265  -7.6243734 -3.0896897]]
torch: tensor([[ 5.6224, -6.0555, -3.0875]])
onnx: [[ 5.6224437 -6.0554595 -3.0875366]]


In [18]:
from inference import torch_model_runner, onnx_model_runner, infer

infer(params, onnx_model_runner(f"{prefix}/model.onnx"), 
      "Отмечается что 44-летняя россиянка первый человек которому предъявят обвинение в попытке вмешаться в выборы 2018 года По информации Минюста она и еще несколько человек выдавала себя за американскую политическую активистку и скрывала свое российское происхождение")
# print(infer(torch_model_runner(model), "Я пришел домой Мама сварила суп"))
# https://onnxruntime.ai/docs/reference/ort-format-models.html

# infer(onnx_model_runner("model.v1.onnx"), "Я пришел домой Мама сварила суп")


'Отмечается, что 44-летняя россиянка, первый человек, которому предъявят обвинение в попытке вмешаться в выборы 2018 года. По информации Минюста она и еще несколько человек выдавала себя за американскую политическую активистку и скрывала свое российское происхождение.'

In [21]:
infer(params, onnx_model_runner(f"{prefix}/model.onnx"), 
      text)


'Мама обычно холодная, как оконное стекло, была в такой ярости, что переколотила дома всю посуду. Хотела даже просить расчёт, но опомнилась: в других ресторанах и своих певиц хватало. Потому мама просто перестала брать Саню с собой, и он впервые оказался предоставлен самому себе. Поначалу Саня робел, один выйти наружу, скучал, слонялся, не зная, чем заняться, — то по комнатам, то по двору. Но к весне осмелел настолько, что исследовал сперва окрестные улицы, а потом и весь обитаемый воронежский мир. Мир этот оказался полон женщинами. Саня, разинув от восхищения рот, смотрел на проплывающих по Большой Дворянской разодетых дам, похожих на вазы с фруктами и цветами. На гимназисток, вечно державшихся дрожащими стайками, словно мотыльки. На горничных, девушек, спешащих по хозяйкиным делам и прошивающих город мелкими аккуратными стежками. Саня влюбился во всех них сразу — и навсегда.'

In [49]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
morph.parse("фывфывфы")[0].inflect({'gent'})

Parse(word='фывфывфов', tag=OpencorporaTag('NOUN,inan,masc plur,gent'), normal_form='фывфывф', score=1.0, methods_stack=((FakeDictionary(), 'фывфывфов', 34, 7), (KnownSuffixAnalyzer(min_word_length=4, score_multiplier=0.5), 'фы')))

In [11]:
# text = """
# Кристина Герасимова (к концу десятилетия, кажется, поменявшая фамилию на Луна) к концу 2010-х оказалась одним из самых работоспособных музыкантов в русской поп-музыке (впрочем, и не только поп), выйдя на уровень, при котором она смогла себе позволить выпускать по альбому в год без потерь в качестве. Лучший из них, впрочем, «Заколдованные сны», как будто бы нарушал классическую для нее схему и уходил то в чистый и практически бессловесный электропоп, то в ностальгию по неприхотливой русской поп-музыке нулевых, то в постпанк — при этом всегда оставаясь песнями самой Луны.
# """
text = """

Мама, обычно холодная, как оконное стекло, была в такой ярости, что переколотила дома всю посуду. Хотела даже просить расчёт, но опомнилась: в других ресторанах и своих певиц хватало. Потому мама просто перестала брать Саню с собой, и он впервые оказался предоставлен самому себе. Поначалу Саня робел один выйти наружу, скучал, слонялся, не зная, чем заняться, — то по комнатам, то по двору. Но к весне осмелел настолько, что исследовал сперва окрестные улицы, а потом и весь обитаемый воронежский мир.

Мир этот оказался полон женщинами.

Саня, разинув от восхищения рот, смотрел на проплывающих по Большой Дворянской разодетых дам, похожих на вазы с фруктами и цветами. На гимназисток, вечно державшихся дрожащими стайками, словно мотыльки. На горничных девушек, спешащих по хозяйкиным делам и прошивающих город мелкими аккуратными стежками.

Саня влюбился во всех них сразу — и навсегда."""
#text = text.replace("\n", " ")
# print(text)

# text = text.replace(".", "").replace(",", "").replace("\n", " ")
# infer(params, onnx_model_runner(f"{prefix}/model.onnx"), text)

In [12]:
from razdel import tokenize
list(map(lambda i: i.text, tokenize(text)))

['Мама',
 ',',
 'обычно',
 'холодная',
 ',',
 'как',
 'оконное',
 'стекло',
 ',',
 'была',
 'в',
 'такой',
 'ярости',
 ',',
 'что',
 'переколотила',
 'дома',
 'всю',
 'посуду',
 '.',
 'Хотела',
 'даже',
 'просить',
 'расчёт',
 ',',
 'но',
 'опомнилась',
 ':',
 'в',
 'других',
 'ресторанах',
 'и',
 'своих',
 'певиц',
 'хватало',
 '.',
 'Потому',
 'мама',
 'просто',
 'перестала',
 'брать',
 'Саню',
 'с',
 'собой',
 ',',
 'и',
 'он',
 'впервые',
 'оказался',
 'предоставлен',
 'самому',
 'себе',
 '.',
 'Поначалу',
 'Саня',
 'робел',
 'один',
 'выйти',
 'наружу',
 ',',
 'скучал',
 ',',
 'слонялся',
 ',',
 'не',
 'зная',
 ',',
 'чем',
 'заняться',
 ',',
 '—',
 'то',
 'по',
 'комнатам',
 ',',
 'то',
 'по',
 'двору',
 '.',
 'Но',
 'к',
 'весне',
 'осмелел',
 'настолько',
 ',',
 'что',
 'исследовал',
 'сперва',
 'окрестные',
 'улицы',
 ',',
 'а',
 'потом',
 'и',
 'весь',
 'обитаемый',
 'воронежский',
 'мир',
 '.',
 'Мир',
 'этот',
 'оказался',
 'полон',
 'женщинами',
 '.',
 'Саня',
 ',',
 'рази

NameError: name 'model' is not defined

In [55]:
from time import time

start = time()
reps = 100
for i in range(0, reps):
    infer(onnx_model_runner("model.v1.onnx"), text)
end = time()

print(1 / ((end - start) / reps / len(text.split(' '))) / 8, 'tokens per second')



172.7963150459843 tokens per second


In [13]:
from time import time
dummy_input = torch.ones((1, params['INPUT_WORDS_CNT'], params['TOTAL_WORD_FEATURES_CNT']))
model.eval()

start = time()
reps = 10000
for i in range(0, reps):
    with torch.no_grad():
        model(dummy_input)
end = time()

print(1 / ((end - start) / reps) / 8, 'tokens per second')

129.86710276061967 tokens per second


In [14]:
from utils import count_parameters
count_parameters(model)

0.19739913940429688

In [3]:
import torch
model = torch.load("model.v1")
model

AttributeError: Can't get attribute 'Model' on <module '__main__'>

In [57]:
import numpy as np
import random
import pymorphy3
import numpy as np
import math
import pickle
from razdel import tokenize

from dataset_builder import calculate_word_features_for_tokens, PAD_TOKEN,get_word_features
from inference import torch_model_runner, onnx_model_runner, infer

onnx_model = onnx_model_runner("results writers big/model.onnx")
with open("params.pickle", "rb") as f:
    params = pickle.load(f)

class jsinfer:
    async def infer(arr):
        class wrapper:
            def to_py():
                return onnx_model(arr)
        return wrapper


async def infer(text):
    assert params["RETAIN_LEFT_PUNCT"] #

    #unpadded_tokens = text.split(' ')
    unpadded_tokens = list(map(lambda i: i.text, tokenize(text)))
    unpadded_tokens = list(filter(lambda x: len(x) > 0, unpadded_tokens))
    tokens = [PAD_TOKEN] * params['INPUT_WORDS_CNT_LEFT'] + unpadded_tokens + [PAD_TOKEN] * (params["INPUT_WORDS_CNT_RIGHT"] + 1)
    features = calculate_word_features_for_tokens(tokens, params)

    res = ""

    i = params['INPUT_WORDS_CNT_LEFT']
    while i < len(tokens) - params['INPUT_WORDS_CNT_RIGHT']:
        tokens_for_batch = tokens[i - params['INPUT_WORDS_CNT_LEFT']: i + params['INPUT_WORDS_CNT_RIGHT']]

        tokens_for_batch_copy = tokens_for_batch.copy()
        tokens_for_batch_copy.insert(params['INPUT_WORDS_CNT_LEFT'], '?')
        # print(" ".join(tokens_for_batch_copy))


        features_for_batch = features[i - params['INPUT_WORDS_CNT_LEFT']: i + params['INPUT_WORDS_CNT_RIGHT']]
        features_for_batch = np.stack((features_for_batch, ))
        arr = np.ascontiguousarray(features_for_batch, dtype=np.float32)
        output_probas = np.array((await jsinfer.infer(arr)).to_py())
        punct_idx = np.argmax(output_probas).item()
        punct = params["ID_TO_PUNCTUATION"][punct_idx]

        #tokens_for_batch_copy[params['INPUT_WORDS_CNT_LEFT']] =  f'<{tokens[i-1], punct, tokens[i]}>'
        tokens_for_batch_copy[params['INPUT_WORDS_CNT_LEFT']] =  f'<{punct}>'
        print(" ".join(tokens_for_batch_copy))

        # print(punct)
        # punct = '.'

        def add_i_token_to_res(i):
            nonlocal res
            if tokens[i] != 'PAD':
                if tokens[i] in '.,:!?':
                    res += tokens[i]
                else:
                    res += " " + tokens[i]

        def insert_punct(punct):
            nonlocal features
            tokens.insert(i, punct)
            features = np.concatenate((features[:i],
                    np.stack((get_word_features(punct, params), )),
                    features[i:]), 0)

        def replace_i_token_punct(punct):
            nonlocal features
            tokens[i] = punct
            features[i] = get_word_features(punct, params)

        def erase_i_token_punct():
            nonlocal features
            del tokens[i]
            features = np.concatenate((features[:i], features[i + 1:]), 0)

        if tokens[i] in '.,!?':
            if tokens[i] == punct or tokens[i] in "!?":# and punct in "."):
                add_i_token_to_res(i)
                add_i_token_to_res(i + 1)
                i += 2
            else:
                if punct != "$empty":
                    res += punct
                    replace_i_token_punct(punct)
                    i += 2
                else:
                    # res += " "
                    erase_i_token_punct()
                    # i += 0

        else:
            if punct != "$empty":
                res += punct #"{ins: " + punct + "`" + tokens[i] + "`}"
                add_i_token_to_res(i)
                insert_punct(punct)
                i += 2
            else:
                add_i_token_to_res(i)
                i += 1

    return res.strip()

await infer("Привет! Как дела? Все хорошо.")

PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD <$empty> Привет ! Как дела ? Все хорошо . PAD PAD PAD PAD PAD PAD PAD PAD
PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD Привет <.> ! Как дела ? Все хорошо . PAD PAD PAD PAD PAD PAD PAD PAD PAD
PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD Привет ! Как <$empty> дела ? Все хорошо . PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD
PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD Привет ! Как дела <.> ? Все хорошо . PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD
PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD Привет ! Как дела ? Все <$empty> хорошо . PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD
PAD PAD PAD PAD PAD PAD PAD PAD PAD Привет ! Как дела ? Все хорошо <.> . PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD


'Привет! Как дела? Все хорошо.'